In [4]:
import io
from typing import Dict, List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pulp
import streamlit as st

In [5]:
def generate_parameters(
    a: int, b: int, G: List[str], seed: int = 0
) -> Tuple[
    Dict[str, int],
    Dict[Tuple[str, int, int, int, int], float],
    Dict[Tuple[int, int, int, int], float],
]:
    """
    パラメータを生成する関数

    Parameters:
        a (int): グリッドの行数
        b (int): グリッドの列数
        G (list): ゴキブリの群れのリスト
        seed (int, optional): 乱数生成の種。デフォルトは0

    Returns:
        tuple: 生成されたパラメータを含むタプル (N_g, V_g, D)
            - N_g (dict): 各群れに属するゴキブリの数
            - V_g (dict): ゴキブリの移動確率
            - D (dict): 殺虫剤に誘引される確率
    """
    np.random.seed(seed)

    # N_gの生成
    N_g = {g: np.random.randint(3, 10) for g in G}

    # V_gとDの乱数生成
    # V_gの生成: 距離に基づいて計算する
    V_g = {}
    for g in G:
        for i in range(a):
            for j in range(b):
                for i_prime in range(a):
                    for j_prime in range(b):
                        distance = np.sqrt((i - i_prime) ** 2 + (j - j_prime) ** 2)
                        V_g[(g, i, j, i_prime, j_prime)] = np.exp(-distance)

    # Dの生成
    D = {}
    for i in range(a):
        for j in range(b):
            for i_prime in range(a):
                for j_prime in range(b):
                    distance = np.sqrt((i - i_prime) ** 2 + (j - j_prime) ** 2)
                    D[(i, j, i_prime, j_prime)] = np.exp(-distance)

    return N_g, V_g, D

In [6]:
a, b = 5, 5
G = ["A", "B", "C"]
N_g, V_g, D = generate_parameters(a, b, G)
V_g

{('A', 0, 0, 0, 0): 1.0,
 ('A', 0, 0, 0, 1): 0.36787944117144233,
 ('A', 0, 0, 0, 2): 0.1353352832366127,
 ('A', 0, 0, 0, 3): 0.049787068367863944,
 ('A', 0, 0, 0, 4): 0.01831563888873418,
 ('A', 0, 0, 1, 0): 0.36787944117144233,
 ('A', 0, 0, 1, 1): 0.2431167344342142,
 ('A', 0, 0, 1, 2): 0.10687792566038574,
 ('A', 0, 0, 1, 3): 0.04232921962320499,
 ('A', 0, 0, 1, 4): 0.016194143319162562,
 ('A', 0, 0, 2, 0): 0.1353352832366127,
 ('A', 0, 0, 2, 1): 0.10687792566038574,
 ('A', 0, 0, 2, 2): 0.059105746561956225,
 ('A', 0, 0, 2, 3): 0.02717246117223556,
 ('A', 0, 0, 2, 4): 0.01142289099346694,
 ('A', 0, 0, 3, 0): 0.049787068367863944,
 ('A', 0, 0, 3, 1): 0.04232921962320499,
 ('A', 0, 0, 3, 2): 0.02717246117223556,
 ('A', 0, 0, 3, 3): 0.01436959609043909,
 ('A', 0, 0, 3, 4): 0.006737946999085467,
 ('A', 0, 0, 4, 0): 0.01831563888873418,
 ('A', 0, 0, 4, 1): 0.016194143319162562,
 ('A', 0, 0, 4, 2): 0.01142289099346694,
 ('A', 0, 0, 4, 3): 0.006737946999085467,
 ('A', 0, 0, 4, 4): 0.003493

In [12]:
obj_1 = {(g, i, j): 1 for g in G for i in range(a) for j in range(b)}
for g in G:
    for i in range(a):
        for j in range(b):
            for i_prime in range(a):
                for j_prime in range(b):
                    obj_1[(g, i, j)] *= (
                        1 - V_g[(g, i, j, i_prime, j_prime)] * D[(i, j, i_prime, j_prime)]
                    )
obj_1

{('A', 0, 0): 0.0,
 ('A', 0, 1): 0.0,
 ('A', 0, 2): 0.0,
 ('A', 0, 3): 0.0,
 ('A', 0, 4): 0.0,
 ('A', 1, 0): 0.0,
 ('A', 1, 1): 0.0,
 ('A', 1, 2): 0.0,
 ('A', 1, 3): 0.0,
 ('A', 1, 4): 0.0,
 ('A', 2, 0): 0.0,
 ('A', 2, 1): 0.0,
 ('A', 2, 2): 0.0,
 ('A', 2, 3): 0.0,
 ('A', 2, 4): 0.0,
 ('A', 3, 0): 0.0,
 ('A', 3, 1): 0.0,
 ('A', 3, 2): 0.0,
 ('A', 3, 3): 0.0,
 ('A', 3, 4): 0.0,
 ('A', 4, 0): 0.0,
 ('A', 4, 1): 0.0,
 ('A', 4, 2): 0.0,
 ('A', 4, 3): 0.0,
 ('A', 4, 4): 0.0,
 ('B', 0, 0): 0.0,
 ('B', 0, 1): 0.0,
 ('B', 0, 2): 0.0,
 ('B', 0, 3): 0.0,
 ('B', 0, 4): 0.0,
 ('B', 1, 0): 0.0,
 ('B', 1, 1): 0.0,
 ('B', 1, 2): 0.0,
 ('B', 1, 3): 0.0,
 ('B', 1, 4): 0.0,
 ('B', 2, 0): 0.0,
 ('B', 2, 1): 0.0,
 ('B', 2, 2): 0.0,
 ('B', 2, 3): 0.0,
 ('B', 2, 4): 0.0,
 ('B', 3, 0): 0.0,
 ('B', 3, 1): 0.0,
 ('B', 3, 2): 0.0,
 ('B', 3, 3): 0.0,
 ('B', 3, 4): 0.0,
 ('B', 4, 0): 0.0,
 ('B', 4, 1): 0.0,
 ('B', 4, 2): 0.0,
 ('B', 4, 3): 0.0,
 ('B', 4, 4): 0.0,
 ('C', 0, 0): 0.0,
 ('C', 0, 1): 0.0,
 ('C', 0, 2)

In [ ]:
def solve_cockroach_elimination(
    a: int,
    b: int,
    G: List[str],
    N_g: Dict[str, int],
    V_g: Dict[Tuple[str, int, int, int, int], float],
    D: Dict[Tuple[int, int, int, int], float],
    max_pesticides: int,
) -> Dict[Tuple[int, int], float]:
    """
    ゴキブリの群れの撲滅問題を解くための関数

    Parameters:
        a (int): グリッドの行数
        b (int): グリッドの列数
        G (list): ゴキブリの群れのリスト
        N_g (dict): 各群れに属するゴキブリの数
        V_g (dict): ゴキブリの移動確率
        D (dict): 殺虫剤に誘引される確率
        max_pesticides (int): 設置可能な最大殺虫剤数

    Returns:
        dict: 各セルに設置される殺虫剤の結果を含む辞書
            - (i, j): セル (i, j) における殺虫剤の設置結果 (0 または 1)
    """
    # 決定変数
    W = pulp.LpVariable.dicts(
        "W", [(i, j) for i in range(a) for j in range(b)], 0, 1, pulp.LpBinary
    )

    # 問題の定式化
    prob = pulp.LpProblem("CockroachElimination", pulp.LpMaximize)

    # 目的関数
    objective_terms = []
    for g in G:
        inner_sum_terms = []
        for i in range(a):
            for j in range(b):
                for i_prime in range(a):
                    for j_prime in range(b):
                        inner_sum_terms.append(
                            1 - V_g[(g, i_prime, j_prime)] * D[(i, j, i_prime, j_prime)]
                        )
        objective_terms.append(N_g[g] * (1 - pulp.lpSum(inner_sum_terms)))

    objective = pulp.lpSum(objective_terms)
    prob += objective

    # 必要な制約条件を追加
    prob += pulp.lpSum([W[(i, j)] for i in range(a) for j in range(b)]) <= max_pesticides

    # 問題を解く
    prob.solve()

    # 結果を取得
    result = {(i, j): W[(i, j)].varValue for i in range(a) for j in range(b)}

    return result


def visualize_result(a: int, b: int, result: Dict[Tuple[int, int], float]) -> io.BytesIO:
    """
    結果を可視化する関数

    Parameters:
        a (int): グリッドの行数
        b (int): グリッドの列数
        result (dict): 各セルに設置される殺虫剤の結果を含む辞書
            - (i, j): セル (i, j) における殺虫剤の設置結果 (0 または 1)

    Returns:
        io.BytesIO: 生成されたグラフ画像を含むバッファ
    """
    fig, ax = plt.subplots()
    for i in range(a):
        for j in range(b):
            if result[(i, j)] == 1:
                ax.scatter(j, -i, color="red", s=100)
    ax.set_xlim(-1, b)
    ax.set_ylim(-a, 1)
    ax.set_aspect("equal")

    # 画像をストリームリットで表示するために保存
    buf = io.BytesIO()
    plt.savefig(buf, format="png")
    buf.seek(0)
    return buf


def main():
    st.title("ゴキブリの群れの撲滅問題")

    # パラメータの設定
    a = st.slider("グリッドの行数", 1, 10, 5)
    b = st.slider("グリッドの列数", 1, 10, 5)
    G = ["A", "B", "C"]
    N_g, V_g, D = generate_parameters(a, b, G)
    max_pesticides = st.slider("設置可能な最大殺虫剤数", 1, 10, 5)

    # 問題の解決
    result = solve_cockroach_elimination(a, b, G, N_g, V_g, D, max_pesticides)

    # 結果の可視化
    buf = visualize_result(a, b, result)
    st.image(buf)


if __name__ == "__main__":
    main()